In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [8]:
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/dataset_adjusted.csv?token=GHSAT0AAAAAACC4ZCNLSI7IUHBG7JHPMLT2ZGMCCBQ"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'dataset_adjusted' variable
    dataset_adjusted = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'dataset_adjusted' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")
    
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/raw/dataset_words.csv?token=GHSAT0AAAAAACC4ZCNLWWASZQU7N7X4D4HOZGMCCXQ"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'words' variable
    words = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'words' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")

Dataset downloaded and loaded into 'dataset_adjusted' successfully.
Dataset downloaded and loaded into 'words' successfully.


In [9]:
# Concatenate the vectorized DataFrame with the original dataset
full_dataset = pd.concat([dataset_adjusted, words], axis=1)

In [5]:
full_dataset.describe()

,Difference,Increase,Decrease,Level,CPI,Unemployment Rate,Consumer Sentiment,Retail Sales,Durable Goods Orders,Short-Term Treasury Bond Rate,...,zlate,zobel,zobel2,zobel3,zobel7,zoebel1,zone,ï½,òscar,şahin
count,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,...,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,0.020833,0.067708,0.046875,2.511458,209.139392,5.703333,86.814167,330463.425000,202291.212500,2.266000,...,0.008333,0.129167,0.016667,0.012500,0.004167,0.004167,0.062500,0.579167,0.012500,0.004167
std,0.225432,0.152897,0.145147,2.160438,40.571082,1.797671,13.265728,109067.410841,35875.582083,2.067154,...,0.091096,0.371561,0.128287,0.111335,0.064550,0.064550,0.843539,3.587808,0.111335,0.064550
min,-1.000000,0.000000,0.000000,0.250000,144.200000,3.400000,50.000000,160605.000000,126529.000000,0.010000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.250000,174.050000,4.400000,77.450000,250207.250000,173829.250000,0.135000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.750000,212.053500,5.300000,89.350000,322071.000000,203816.500000,1.690000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,4.812500,237.740000,6.225000,96.025000,394967.500000,228374.500000,4.482500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.750000,0.750000,1.000000,6.500000,302.918000,14.700000,112.000000,603211.000000,297311.000000,6.190000,...,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,13.000000,32.000000,1.000000,1.000000


In [10]:
full_dataset[Date]

NameError: name 'Date' is not defined